In [58]:
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [59]:
data = pd.read_csv('./All_States_GE.csv')

C:\Users\Rahul\AppData\Local\Temp\ipykernel_17824\3516144033.py:1: DtypeWarning: Columns (17,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./All_States_GE.csv')


In [60]:
# subset df statename, party
subsets = ['State_Name', 'Sex']
subsetDf = data[subsets]
subsetDf = subsetDf.dropna(subset=subsets)
subsetDf

,State_Name,Sex
0,Andaman_&_Nicobar_Islands,M
1,Andaman_&_Nicobar_Islands,M
2,Andaman_&_Nicobar_Islands,M
3,Andaman_&_Nicobar_Islands,M
4,Andaman_&_Nicobar_Islands,M
...,...,...
91646,Uttar_Pradesh,F
91651,Andhra_Pradesh,M
91657,Mysore,M
91658,Mysore,M


In [61]:
sex_mapping = {"M": 0, "O": 0, "F": 1, "NOTA": 2, "male": 0, "MALE": 0, "female": 1, "FEMALE": 1} 
subsetDf2 = subsetDf.copy()
subsetDf2["SInt"] = subsetDf["Sex"].map(sex_mapping)

subsetDf2 = subsetDf2[subsetDf2.SInt != 2]
subsetDf2.drop('Sex', axis=1, inplace=True)
subsetDf2


,State_Name,SInt
0,Andaman_&_Nicobar_Islands,0
1,Andaman_&_Nicobar_Islands,0
2,Andaman_&_Nicobar_Islands,0
3,Andaman_&_Nicobar_Islands,0
4,Andaman_&_Nicobar_Islands,0
...,...,...
91646,Uttar_Pradesh,1
91651,Andhra_Pradesh,0
91657,Mysore,0
91658,Mysore,0


In [62]:

tobeEncoded = [x for x in subsetDf2.columns if subsetDf2[x].dtype == 'object']
tobeEncoded

['State_Name']

In [63]:
encoder = preprocessing.LabelEncoder()

for x in tobeEncoded:
    subsetDf2[x] = encoder.fit_transform(subsetDf2[x])

# count unique values
subsetDf2["SInt"].value_counts()

# Equalize the number of samples for each class
subsetDf2 = subsetDf2.groupby('SInt').apply(lambda x: x.sample(subsetDf2['SInt'].value_counts().min()).reset_index(drop=True))
subsetDf2["SInt"].value_counts()
subsetDf2


State_Name  SInt
SInt                       
0    0             37     0
     1             39     0
     2             14     0
     3             21     0
     4             18     0
...               ...   ...
1    4735           4     1
     4736          31     1
     4737          13     1
     4738          29     1
     4739          29     1

[9480 rows x 2 columns]

In [64]:
# We are predicint SInt

X = subsetDf2.drop('SInt', axis=1)
Y = subsetDf2['SInt']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
subsetDf2

State_Name  SInt
SInt                       
0    0             37     0
     1             39     0
     2             14     0
     3             21     0
     4             18     0
...               ...   ...
1    4735           4     1
     4736          31     1
     4737          13     1
     4738          29     1
     4739          29     1

[9480 rows x 2 columns]

In [65]:
from sklearn.metrics import accuracy_score
# Make model

model = LogisticRegression()
model.fit(X_train, Y_train)

# Test model

predictions = model.predict(X_test)
print(accuracy_score(Y_test, predictions))
print(list(predictions))

0.495253164556962
[0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 